In [18]:
from langchain.vectorstores.chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
import time, os, shutil
CHROMA_PATH = "final_test/chroma_test"
CHROMA_PATH = "final_test/chroma_test2"

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

def split_text(documents: list[Document]):
    print("Starting chunking")

    "Recursive Splitter"
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=300,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")
    print(f"len docs {len(chunks)}")
    document = chunks[10]
    print("page content \n", document.page_content)
    print("doc metadata \n", document.metadata)
    return chunks

In [20]:
def load_embedding_model():
    start_time = time.time()
    model_path="intfloat/multilingual-e5-large"   
    encode_kwargs = {"normalize_embeddings": True}
    local_embedding = HuggingFaceEmbeddings(
        model_name=model_path,
        cache_folder="./models",
        encode_kwargs=encode_kwargs
    )
    end_time = time.time()
    print(f'model load time {round(end_time - start_time, 0)} second')
    return local_embedding

embedding = load_embedding_model()

d:\Python\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


model load time 132.0 second


In [4]:
def save_to_chroma(chunks: list[Document]):
    print("Starting Embedding")
    # Clear out the database first.
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

    # Create a new DB from the documents.
    start_time = time.time()
    db = Chroma.from_documents(
        chunks, 
        embedding, 
        persist_directory=CHROMA_PATH,
        collection_metadata={"hnsw:space": "cosine"}
    )
    db.persist()
    end_time = time.time()
    print(f'embedding time {round(end_time - start_time, 0)} second')
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

# Nutrisi pada anak dengan penyakit ginjal

### unstructured

In [4]:
from unstructured.partition.text import partition_text

elements = partition_text(filename="final_clean/nutrisi_pada_anak_dengan_penyakit_ginjal_clean_poster_final.txt")

In [16]:
from unstructured.chunking.basic import chunk_elements

chunks = chunk_elements(elements, 
                        max_characters=1500, 
                        new_after_n_chars=1000, 
                        overlap=300, 
                        overlap_all=True)

In [33]:
chunks[0].metadata.to_dict()

{'file_directory': 'final_clean',
 'filename': 'nutrisi_pada_anak_dengan_penyakit_ginjal_clean_poster_final.txt',
 'filetype': 'text/plain',
 'languages': ['ind'],
 'last_modified': '2024-04-28T12:28:39'}

In [35]:
document_chunks = []
for chunk in chunks:
    temp_metadata = chunk.metadata.to_dict()
    temp_metadata["languages"] = 'ind'
    temp = Document(
            page_content=chunk.text,
            metadata=temp_metadata,
        )
    document_chunks.append(temp)

In [ ]:
# unstructured
db = Chroma.from_documents(
        document_chunks, 
        embedding, 
        persist_directory=CHROMA_PATH, 
        collection_metadata={"hnsw:space": "cosine"}
    )
db.persist()

### langchain

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("final_clean/nutrisi_pada_anak_dengan_penyakit_ginjal_clean_poster_final.txt")
documents = loader.load()

In [ ]:
# langchain splitter
chunks = split_text(documents)
db = Chroma.from_documents(
        chunks, 
        embedding, 
        persist_directory=CHROMA_PATH, 
        collection_metadata={"hnsw:space": "cosine"}
    )
db.persist()

### tabel

In [10]:
from unstructured.partition.docx import partition_docx

elements = partition_docx(filename="docx/nutrisi tabel.docx")

In [12]:
elements[0].metadata.to_dict()

{'category_depth': 0,
 'emphasized_text_contents': ['Tabel 1'],
 'emphasized_text_tags': ['b'],
 'last_modified': '2024-05-02T00:11:11',
 'page_number': 1,
 'languages': ['ind'],
 'file_directory': 'docx',
 'filename': 'nutrisi tabel.docx',
 'filetype': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document'}

In [13]:
import re

text = ""
for elem in elements:
    text += elem.text
print(text)
table_list = re.split(r"Tabel \d", text)
table_list.pop(0)

Tabel 1Klasifikasi derajat penyakit ginjal kronik.Derajat 1: Kerusakan ginjal dengan LFG normal atau meningkat, GFR 60-89 mL/min/1.73 m².Derajat 2: Kerusakan ginjal dengan penurunan ringan LFG, GFR 60-89 mL/min/1.73 m².Derajat 3: Penurunan sedang LFG, GFR 30-59 mL/min/1.73 m².Derajat 4: Penurunan berat LFG,  GFR 15-29 mL/min/1.73 m².Derajat 5: gagal ginjal,  GFR <15 mL/min/1.73 m²atau dialisis.Tabel 2 Rekomendasi parameter dan frekuensi penilaian nutrisi untuk anak dengan PGK derajat 2 sampai 5 dan 5-dialisis (5D)Umur < 1 tahun, PGK 2 – 3, Asupan makanan : 0,5 – 3 bulan, Persentil tinggi badan terhadap umur : 0,5 – 1,5 bulan, Persentil velocity tinggi badan terhadap umur : 0,5 – 2 bulan, Perkiraan berat kering dan persentil berat badan terhadap umur : 0,5 – 1,5 bulan, Indeks massa tubuh : 0,5 – 1,5 bulan, Persentil lingkar kepala : 0,5 – 1,5 bulanUmur < 1 tahun, PGK 4 – 5, Asupan makanan : 0,5 – 3 bulan, Persentil tinggi badan terhadap umur : 0,5 – 1,5 bulan, Persentil velocity tinggi 

''

In [14]:
table_list

['Klasifikasi derajat penyakit ginjal kronik.Derajat 1: Kerusakan ginjal dengan LFG normal atau meningkat, GFR 60-89 mL/min/1.73 m².Derajat 2: Kerusakan ginjal dengan penurunan ringan LFG, GFR 60-89 mL/min/1.73 m².Derajat 3: Penurunan sedang LFG, GFR 30-59 mL/min/1.73 m².Derajat 4: Penurunan berat LFG,  GFR 15-29 mL/min/1.73 m².Derajat 5: gagal ginjal,  GFR <15 mL/min/1.73 m²atau dialisis.',
 ' Rekomendasi parameter dan frekuensi penilaian nutrisi untuk anak dengan PGK derajat 2 sampai 5 dan 5-dialisis (5D)Umur < 1 tahun, PGK 2 – 3, Asupan makanan : 0,5 – 3 bulan, Persentil tinggi badan terhadap umur : 0,5 – 1,5 bulan, Persentil velocity tinggi badan terhadap umur : 0,5 – 2 bulan, Perkiraan berat kering dan persentil berat badan terhadap umur : 0,5 – 1,5 bulan, Indeks massa tubuh : 0,5 – 1,5 bulan, Persentil lingkar kepala : 0,5 – 1,5 bulanUmur < 1 tahun, PGK 4 – 5, Asupan makanan : 0,5 – 3 bulan, Persentil tinggi badan terhadap umur : 0,5 – 1,5 bulan, Persentil velocity tinggi badan t

In [16]:
from langchain_core.documents import Document

document_chunks = []
for table in table_list:
    temp = Document(
            page_content=table,
            metadata={'filename':'nutrisi_pada_anak_dengan_penyakit_ginjal.pdf'}
        )
    document_chunks.append(temp)

In [21]:
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding)
db.add_documents(document_chunks)

['0672aa08-084f-11ef-ad4c-b6b024e70617',
 '0672aa09-084f-11ef-9189-b6b024e70617',
 '0672aa0a-084f-11ef-b700-b6b024e70617',
 '0672aa0b-084f-11ef-9b66-b6b024e70617',
 '0672aa0c-084f-11ef-bac4-b6b024e70617',
 '0672aa0d-084f-11ef-868f-b6b024e70617',
 '0672aa0e-084f-11ef-a2bb-b6b024e70617',
 '0672aa0f-084f-11ef-9db0-b6b024e70617']

# Tatalaksana Hemodialisis pada Anak dan Bayi

### Unstructured

In [37]:
elements = partition_text(filename="final_clean/Tatalaksana_Hemodialisis_pada_Anak_dan_Bayi_clean_test.txt")

In [ ]:
chunks = chunk_elements(elements, 
                        max_characters=1500, 
                        new_after_n_chars=1000, 
                        overlap=300, 
                        overlap_all=True)

In [ ]:
document_chunks = []
for chunk in chunks:
    temp_metadata = chunk.metadata.to_dict()
    temp_metadata["languages"] = 'ind'
    temp = Document(
            page_content=chunk.text,
            metadata=temp_metadata,
        )
    document_chunks.append(temp)

In [ ]:
db.add_documents(document_chunks)

### langchain

In [8]:
loader = TextLoader("final_clean/Tatalaksana_Hemodialisis_pada_Anak_dan_Bayi_clean_test.txt", encoding='utf-8')
documents = loader.load()

In [9]:
chunks = split_text(documents)
db.add_documents(chunks)

Starting chunking
Split 1 documents into 28 chunks.
len docs 28
page content 
 dewasa dan pediatrik. Misalnya, pada pediatrik, aliran darah dan hemodialyzer dipilih atas dasar bahwa anak-anak dapat mentolerir 8% (maksimum 10%) total volume darah mereka di sirkuit ekstrakorporeal berdasarkan perkiraan volume darah total 80 mL/ kg untuk bayi dan 70 mL/ kg untuk anak yang lebih tua. Jika sirkuit hemodialisis terkecil yang tersedia melebihi volume kritis ini, dapat ditutupi dengan larutan albumin manusia 4,5% atau darah donor untuk mencegah gejala hipovolemia.12,14 Karena aliran darah bayi dan anak memiliki kaliber lebih kecil, kecepatan pompa darah cenderung lebih cepat, dengan sasaran 8–10 mL/kg/menit, dibandingkan 3-5 mL/kg/menit untuk hemodialisis dewasa. Penggulung kepala pompa darah (blood pump head rollers) harus disesuaikan dengan ukuran aliran dialisis, hemolisis mekanik dapat terjadi jika aliran darah dewasa digunakan dalam mesin dialisis yang disiapkan untuk saluran pediatrik.12

['ed96aad5-0525-11ef-90e0-b6b024e70617',
 'ed96aad6-0525-11ef-abd2-b6b024e70617',
 'ed96aad7-0525-11ef-acdb-b6b024e70617',
 'ed96aad8-0525-11ef-837a-b6b024e70617',
 'ed96aad9-0525-11ef-b452-b6b024e70617',
 'ed96aada-0525-11ef-9e79-b6b024e70617',
 'ed96aadb-0525-11ef-ac62-b6b024e70617',
 'ed96aadc-0525-11ef-83eb-b6b024e70617',
 'ed96aadd-0525-11ef-8bdf-b6b024e70617',
 'ed96aade-0525-11ef-997e-b6b024e70617',
 'ed96aadf-0525-11ef-a0cb-b6b024e70617',
 'ed96aae0-0525-11ef-9a1b-b6b024e70617',
 'ed96aae1-0525-11ef-a3b2-b6b024e70617',
 'ed96aae2-0525-11ef-900a-b6b024e70617',
 'ed96d1e8-0525-11ef-a656-b6b024e70617',
 'ed96d1e9-0525-11ef-b720-b6b024e70617',
 'ed96d1ea-0525-11ef-a4e5-b6b024e70617',
 'ed96d1eb-0525-11ef-8d86-b6b024e70617',
 'ed96d1ec-0525-11ef-aeb1-b6b024e70617',
 'ed96d1ed-0525-11ef-a026-b6b024e70617',
 'ed96d1ee-0525-11ef-8594-b6b024e70617',
 'ed96d1ef-0525-11ef-9f3e-b6b024e70617',
 'ed96d1f0-0525-11ef-914a-b6b024e70617',
 'ed96d1f1-0525-11ef-a6f4-b6b024e70617',
 'ed96d1f2-0525-

# Tatalaksana Penyakit Ginjal Kronik pada Anak

### unstructured

In [41]:
elements = partition_text(filename="final_clean/Tatalaksana_Penyakit_Ginjal_Kronik_pada_Anak_clean_split_final.txt")

In [ ]:
chunks = chunk_elements(elements, 
                        max_characters=1500, 
                        new_after_n_chars=1000, 
                        overlap=300, 
                        overlap_all=True)

In [ ]:
document_chunks = []
for chunk in chunks:
    temp_metadata = chunk.metadata.to_dict()
    temp_metadata["languages"] = 'ind'
    temp = Document(
            page_content=chunk.text,
            metadata=temp_metadata,
        )
    document_chunks.append(temp)

In [ ]:
db.add_documents(document_chunks)

### langchain

In [10]:
loader = TextLoader("final_clean/Tatalaksana_Penyakit_Ginjal_Kronik_pada_Anak_clean_split_final.txt", encoding='utf-8')
documents = loader.load()

In [11]:
chunks = split_text(documents)
db.add_documents(chunks)

Starting chunking
Split 1 documents into 12 chunks.
len docs 12
page content 
 Penyakit ginjal kronis stadium 5 merupakan indikasi untuk transplantasi. Meskipun demikian, tidak semua 
pasien dengan penyakit ginjal kronis stadium 5 dapat 
menjadi kandidat untuk transplantasi ginjal. Prosedur ini 
dapat terlalu berisiko bagi sebagian karena komorbiditas
yang telah diderita pasien atau karena kontraindikasi 
tertentu, seperti infeksi kronis yang akan dieksaserbasi oleh imunosupresi. Kontraindikasi absolut bagi transplantasi ginjal adalah keganasan aktif, terutama jika telah bermetastasis. Infeksi HIV dan potensi rekurensi penyakit ginjal juga perlu dipertimbangkan 
dalam rencana transplantasi.
Prognosis pasien dengan penyakit ginjal kronis adalah 
bervariasi menurut stadium dan penatalaksanaan yang dilakukan. Dengandeteksi dan penatalaksanaan dini,morbiditas dan mortalitas diharapkan dapat diturunkan.
3.Kesimpulan
Penyakit ginjal kronis pada anak dapat disebabkan penyakit 
kongenital, did

['3be104ae-0526-11ef-b7ab-b6b024e70617',
 '3be104af-0526-11ef-b8b7-b6b024e70617',
 '3be104b0-0526-11ef-a1a2-b6b024e70617',
 '3be104b1-0526-11ef-84a6-b6b024e70617',
 '3be104b2-0526-11ef-b991-b6b024e70617',
 '3be104b3-0526-11ef-98c0-b6b024e70617',
 '3be104b4-0526-11ef-b8af-b6b024e70617',
 '3be104b5-0526-11ef-9178-b6b024e70617',
 '3be104b6-0526-11ef-a6e7-b6b024e70617',
 '3be104b7-0526-11ef-a582-b6b024e70617',
 '3be104b8-0526-11ef-b79e-b6b024e70617',
 '3be104b9-0526-11ef-994e-b6b024e70617']

### tabel

In [4]:
from unstructured.partition.docx import partition_docx

tata_laksana_tabel_elements = partition_docx(filename="docx/tatalaksana anak tabel.docx")

In [24]:
import re

tata_anak_tabel_text = ""
for elem in tata_laksana_tabel_elements:
    tata_anak_tabel_text += elem.text
# print(tata_anak_tabel_text)
table_list = re.split(r"Tabel \d", tata_anak_tabel_text)
table_list.pop(0)

''

In [25]:
table_list

['. Stadium Penyakit ginjal kronik berdasarkan laju filtrasi glomerulusStadium 1 Kerusakan ginjal dengan LFG normal/meningkat (<90 ml/menit/1,73 m2), Pengobatan primer dan kondisi komorbiditas.Stadium 2 Kerusakan ginjal dengan sedikit penurunan LFG (60-89 ml/menit/1,73 m2), Menekan laju kejadian PGKStadium 3 Kerusakan ginjal dengan penurunan sedang LFG (30-59 ml/menit/1,73 m2), Evaluasi dan pengobatan komplikasiStadium 4 Kerusakan ginjal dengan penurunan besar LFG (15-29 ml/menit/1,73 m2), Persiapan transplantasi ginjalStadium 5 Gagal ginjal, LFG < 15 ml/menit/1,73 m2, Terapi pengganti ginjal',
 '. Dosis Obat Hipertensi Pada AnakDiuretikaHidroklorotiazid, dosis oral awal per hari 1mg/kg, dosis oral maksimal per hari 4 mg/kg, interval dosis Tiap 12 jamKlortalidon, dosis oral awal per hari 1 mg/kg, dosis oral maksimal per hari 2 mg/kg, interval dosis Sekali sehariSpironolakton, dosis oral awal per hari 1 mg/kg, dosis oral maksimal per hari 3  mg/kg, interval dosis Tiap 12 jamFurosemid, d

In [26]:
document_chunks = []
for table in table_list:
    temp = Document(
            page_content=table,
            metadata={'filename':'Tatalaksana_Penyakit_Ginjal_Kronik_pada_Anak.pdf'}
        )
    document_chunks.append(temp)

In [27]:
db.add_documents(document_chunks)

['4d04dade-084f-11ef-b9de-b6b024e70617',
 '4d04dadf-084f-11ef-8479-b6b024e70617',
 '4d04dae0-084f-11ef-850c-b6b024e70617',
 '4d04dae1-084f-11ef-9fbb-b6b024e70617',
 '4d04dae2-084f-11ef-9458-b6b024e70617']